<a href="https://colab.research.google.com/github/deborahmasibo/Moringa-Core-Module-2-Week-3-IP/blob/main/Hypothyroidism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>